In [108]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import unicodedata
import pandas as pd

In [2]:
URL = 'https://www.scvotes.gov/how-register-absentee-voting'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
results = soup.find(id="block-vc3-content")

In [4]:
countyLinks = results.find_all("a")

In [5]:
print(countyLinks)

[<a href="/abbeville">Abbeville</a>, <a href="/aiken">Aiken</a>, <a href="/allendale">Allendale</a>, <a href="/anderson">Anderson</a>, <a href="/bamberg">Bamberg</a>, <a href="/barnwell">Barnwell</a>, <a href="/beaufort">Beaufort</a>, <a href="/berkeley">Berkeley</a>, <a href="/calhoun">Calhoun</a>, <a href="/charleston">Charleston</a>, <a href="/cherokee">Cherokee</a>, <a href="/chester">Chester</a>, <a href="/chesterfield">Chesterfield</a>, <a href="/clarendon">Clarendon</a>, <a href="/colleton">Colleton</a>, <a href="/darlington">Darlington</a>, <a href="/dillon">Dillon</a>, <a href="/dorchester">Dorchester</a>, <a href="/edgefield">Edgefield</a>, <a href="/fairfield">Fairfield</a>, <a href="/florence">Florence</a>, <a href="/georgetown">Georgetown</a>, <a href="/greenville">Greenville</a>, <a href="/greenwood">Greenwood</a>, <a href="/hampton">Hampton</a>, <a href="/horry">Horry</a>, <a href="/jasper">Jasper</a>, <a href="/kershaw">Kershaw</a>, <a href="/lancaster">Lancaster</a>, <

In [26]:
countyURLs = {}
for element in countyLinks:
    raw = str(element)
    name = element.text
    indexes=[]
    for i in range(0, len(raw)): 
        if raw[i] == '"': 
            indexes.append(i)
    url = "https://www.scvotes.gov" + raw[(indexes[0]+1): (indexes[1])]
    countyURLs[name] = url

In [27]:
print(countyURLs)

{'Abbeville': 'https://www.scvotes.gov/abbeville', 'Aiken': 'https://www.scvotes.gov/aiken', 'Allendale': 'https://www.scvotes.gov/allendale', 'Anderson': 'https://www.scvotes.gov/anderson', 'Bamberg': 'https://www.scvotes.gov/bamberg', 'Barnwell': 'https://www.scvotes.gov/barnwell', 'Beaufort': 'https://www.scvotes.gov/beaufort', 'Berkeley': 'https://www.scvotes.gov/berkeley', 'Calhoun': 'https://www.scvotes.gov/calhoun', 'Charleston': 'https://www.scvotes.gov/charleston', 'Cherokee': 'https://www.scvotes.gov/cherokee', 'Chester': 'https://www.scvotes.gov/chester', 'Chesterfield': 'https://www.scvotes.gov/chesterfield', 'Clarendon': 'https://www.scvotes.gov/clarendon', 'Colleton': 'https://www.scvotes.gov/colleton', 'Darlington': 'https://www.scvotes.gov/darlington', 'Dillon': 'https://www.scvotes.gov/dillon', 'Dorchester': 'https://www.scvotes.gov/dorchester', 'Edgefield': 'https://www.scvotes.gov/edgefield', 'Fairfield': 'https://www.scvotes.gov/fairfield', 'Florence': 'https://www.

In [110]:
columns_list = ['state', 'county', 'location_type', 'name','address_1', 'address_2', 'city', 'state','zip', 'phone', 'county_website_url', 'email', 'fax']
#[state, county, location_type, name, address_1, address_2, city, state, zip_code, phone, county_website_url, email, fax ]
parsed_Data = []

In [111]:
for county_name, county_url in countyURLs.items():
    #print(county_url)
    page2 = requests.get(county_url)
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    result2 = soup2.find(id="block-vc3-content")
    content = result2.find_all("p")
    #print(content)
    state = "SC"
    county = county_name
    location_type = "Board of Voter Registration"
    name = np.nan
    address_1 = np.nan
    address_2 = np.nan
    city = np.nan
    zip_code = np.nan
    phone = np.nan
    county_website_url = county_url
    validate_url = np.nan
    email = np.nan
    fax = np.nan
    #full_text = []
    for item in content:
        item = unicodedata.normalize('NFKD', item.text)
        #full_text.append(item)
        if "Director" in item:
            try: 
                name = re.search('\w+\s\w+.*Director$', item).group()
            except:
                name = item.strip()
        #Special case for Chesterfield
        elif "Kevin" in item:
            name = item.strip()
        # Regex looking for addresses
        elif re.search('\d+\s\w+.\s\w+.*', item) and "Post" not in item:
            address_1 = item.strip()
        elif "Post" in item or "PO" in item:
            address_2 = item.strip()
        # Regex looking for City, State, Zip
        elif re.search('(\w+[.]*\s*\w*)[,]*\s[SC].*(\d{5})', item):
            query = re.compile('(\w+[.]*\s*\w*)[,]*\s[SC].*(\d{5})')
            m = query.match(item)
            city = m.group(1)
            zip_code = m.group(2)
        # Regex looking for phone number without Fax
        elif re.search('[(]*\d{3}[)]*\s*[-]*\d{3}[-]\d{4}', item) and "Fax" not in item:
            number = item.strip()
            if 'Office' in number:
                number = number.replace('Office', '').strip()
            if 'Phone' in number:
                number = number.replace('Phone', '').strip()
            phone = number.replace(':', '').strip()
        # Regex looking for phone number with Fax
        elif re.search('[(]\d{3}[)]\s*\d{3}[-]\d{4}', item) and "Fax" in item:
            query = re.compile('[(]\d{3}[)]\s*\d{3}[-]\d{4}')
            # Use Search since desired text is not at the start of the string
            m = query.search(item)
            fax = m.group(0)
    # Some sites have multiple url's, breaking out of loop to look at Full Content
    # Regex looking for URL, If found replace Sec State URL with local
    links = []
    for link in result2.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
    #print(links)
    
    if len(links) != 0:
        #query = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
        #m=query.search(' '.join(links))
        #county_website_url = m.group(0)
        county_website_url = links[0]
        
    # Regex looking for email address
    emails = []
    for email_result in result2.findAll('a', attrs={'href': re.compile("^mailto:")}):
        emails.append(email_result.text)
    if len(emails) != 0:
        email = emails[0]
    #print(county_listing)
        
    county_listing = [state, county, location_type, name, address_1, address_2, city, state, zip_code, phone, county_website_url, email, fax ]
    print(county_listing)
    parsed_Data.append(county_listing)

['SC', 'Abbeville', 'Board of Voter Registration', 'Kim London - Director', '903 West Greenwood Street, Suite 2500', nan, 'Abbeville', 'SC', '29620', '(864) 366-2400, ext. 53', 'https://www.scvotes.gov/abbeville', 'klondon@abbevillecountysc.com', '(864) 366-5797']
['SC', 'Aiken', 'Board of Voter Registration', 'Cynthia Holland-Director', '1930 University Parkway', 'Post Office Box 3127', 'Aiken', 'SC', '29802', '(803) 642-2028', 'https://www.scvotes.gov/aiken', 'cholland@aikencountysc.gov', '(803) 642-2029']
['SC', 'Allendale', 'Board of Voter Registration', 'Sonya Prince-Director', '158 McNair Street', nan, 'Allendale', 'SC', '29810', '(803) 584-4178', 'https://www.scvotes.gov/allendale', 'Sprince@allendalecounty.com', '(803) 584-8147']
['SC', 'Anderson', 'Board of Voter Registration', 'Katy Smith- Director', '301 N. Main St.', 'Post Office Box 8002 (29622)', 'Anderson', 'SC', '29621', '(864) 260-4035', 'http://www.andersoncountysc.org/Departments/Registration-Elections', 'acvote@ande

['SC', 'Marlboro', 'Board of Voter Registration', 'Lori Moses -Director', nan, 'Post Office Box 502', 'Bennettsville', 'SC', '29512', '(843) 479-5612', 'https://www.scvotes.gov/marlboro', 'lmoses@marlborocounty.sc.gov', '(843) 454-0633']
['SC', 'McCormick', 'Board of Voter Registration', 'Sojourner Jennings- Director', '610 South Mine Street', 'Post Office Box 636', 'McCormick', 'SC', '29835', '(864) 852-2089', 'https://www.scvotes.gov/mccormick', 'sojennings@mccormickcountysc.org', '(864) 852-2489']
['SC', 'Newberry', 'Board of Voter Registration', 'Shanna Proctor- Director', '1872 Wilson Road', 'Post Office Box 147', 'Newberry', 'SC', '29108', '(803) 321-2121', 'https://www.scvotes.gov/newberry', 'sproctor@newberrycounty.net', '(803) 321-2122']
['SC', 'Oconee', 'Board of Voter Registration', 'Joy Scharich-Director', '415 South Pine Street\n\nWalhalla SC 29691\n\nOffice (864) 638-4196', nan, nan, 'SC', nan, nan, 'https://www.scvotes.gov/oconee', 'jscharich@oconeesc.com', nan]
['SC', '

In [112]:
test = pd.DataFrame(data = parsed_Data, columns = columns_list)
test.head()

,state,county,location_type,name,address_1,address_2,city,state,zip,phone,county_website_url,email,fax
0,SC,Abbeville,Board of Voter Registration,Kim London - Director,"903 West Greenwood Street, Suite 2500",NaN,Abbeville,SC,29620,"(864) 366-2400, ext. 53",https://www.scvotes.gov/abbeville,klondon@abbevillecountysc.com,(864) 366-5797
1,SC,Aiken,Board of Voter Registration,Cynthia Holland-Director,1930 University Parkway,Post Office Box 3127,Aiken,SC,29802,(803) 642-2028,https://www.scvotes.gov/aiken,cholland@aikencountysc.gov,(803) 642-2029
2,SC,Allendale,Board of Voter Registration,Sonya Prince-Director,158 McNair Street,NaN,Allendale,SC,29810,(803) 584-4178,https://www.scvotes.gov/allendale,Sprince@allendalecounty.com,(803) 584-8147
3,SC,Anderson,Board of Voter Registration,Katy Smith- Director,301 N. Main St.,Post Office Box 8002 (29622),Anderson,SC,29621,(864) 260-4035,http://www.andersoncountysc.org/Departments/Re...,acvote@andersoncountysc.org,(864) 260-4203
4,SC,Bamberg,Board of Voter Registration,Melinda C. Johnson -Director,1234 North St.,Post Office Box 947,Bamberg,SC,29003,(803) 245-3028,https://www.scvotes.gov/bamberg,johnsonmc@bambergcounty.sc.gov,(803) 245-7968


In [113]:
test.to_csv('./test.csv')

In [ ]:
result2 = soup2.find(id="block-vc3-content")
print(result2)

In [ ]:
#content = result2.find_all("p")
content = result2.find(class_='content').find_all('p')

In [ ]:
print(content[2].text)